# Decision Tree Classifier

In this notebook, you will implement your own decision tree algorithm for the classification problem. You are supposed to learn:

* How to prepare the dataset for training and testing of the model (i.e. decision tree).
* How to implement the decision tree learning algorithm.
* How to classify unseen samples using your model (i.e. trained decision tree).
* How to evaluate the performance of your model.

**Instructions:**

* Read carefuly through this notebook. Be sure you understand what is provided to you, and what is required from you.
* Place your code/edit only in sections annotated with `### START CODE HERE ###` and `### END CODE HERE ###`.
* Use comments whenever the code is not self-explanatory.
* Submit an executable notebook (`*.ipynb`) with your solution to BlackBoard.

Enjoy :-)

## Packages

Following packages is all you need. Do not import any additional packages!

* [Pandas](https://pandas.pydata.org/) is a library providing easy-to-use data structures and data analysis tools.
* [Numpy](http://www.numpy.org/) library provides support for large multi-dimensional arrays and matrices, along with functions to operate on these.

In [1136]:
import pandas as pd
import numpy as np
import itertools

## Problem

You are given a dataset `mushrooms.csv` with characteristics/attributes of mushrooms, and your task is to implement, train and evaluate a decision tree classifier able to say whether a mushroom is poisonous or edible based on its attributes.

## Dataset

The dataset of mushroom characteristics is freely available at [Kaggle Datasets](https://www.kaggle.com/uciml/mushroom-classification) where you can find further information about the dataset. It consists of 8124 mushrooms characterized by 23 attributes (including the class). Following is the overview of attributes and values:

* class: edible=e, poisonous=p
* cap-shape: bell=b,conical=c,convex=x,flat=f, knobbed=k,sunken=s
* cap-surface: fibrous=f,grooves=g,scaly=y,smooth=s
* cap-color: brown=n,buff=b,cinnamon=c,gray=g,green=r,pink=p,purple=u,red=e,white=w,yellow=y
* bruises: bruises=t,no=f
* odor: almond=a,anise=l,creosote=c,fishy=y,foul=f,musty=m,none=n,pungent=p,spicy=s
* gill-attachment: attached=a,descending=d,free=f,notched=n
* gill-spacing: close=c,crowded=w,distant=d
* gill-size: broad=b,narrow=n
* gill-color: black=k,brown=n,buff=b,chocolate=h,gray=g, green=r,orange=o,pink=p,purple=u,red=e,white=w,yellow=y
* stalk-shape: enlarging=e,tapering=t
* stalk-root: bulbous=b,club=c,cup=u,equal=e,rhizomorphs=z,rooted=r,missing=?
* stalk-surface-above-ring: fibrous=f,scaly=y,silky=k,smooth=s
* stalk-surface-below-ring: fibrous=f,scaly=y,silky=k,smooth=s
* stalk-color-above-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y
* stalk-color-below-ring: brown=n,buff=b,cinnamon=c,gray=g,orange=o,pink=p,red=e,white=w,yellow=y
* veil-type: partial=p,universal=u
* veil-color: brown=n,orange=o,white=w,yellow=y
* ring-number: none=n,one=o,two=t
* ring-type: cobwebby=c,evanescent=e,flaring=f,large=l,none=n,pendant=p,sheathing=s,zone=z
* spore-print-color: black=k,brown=n,buff=b,chocolate=h,green=r,orange=o,purple=u,white=w,yellow=y
* population: abundant=a,clustered=c,numerous=n,scattered=s,several=v,solitary=y
* habitat: grasses=g,leaves=l,meadows=m,paths=p,urban=u,waste=w,woods=d

Let's load the dataset into so called Pandas dataframe.

In [1137]:
mushrooms_df = pd.read_csv('mushrooms.csv')

Now we can take a closer look at the data.

In [1138]:
mushrooms_df

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8120,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8121,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8122,p,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l


You can also print an overview of all attributes with the counts of unique values.

In [1139]:
mushrooms_df.describe().T

,count,unique,top,freq
class,8124,2,e,4208
cap-shape,8124,6,x,3656
cap-surface,8124,4,y,3244
cap-color,8124,10,n,2284
bruises,8124,2,f,4748
odor,8124,9,n,3528
gill-attachment,8124,2,f,7914
gill-spacing,8124,2,c,6812
gill-size,8124,2,b,5612
gill-color,8124,12,b,1728


The dataset is pretty much balanced. That's a good news for the evaluation.

## Dataset Preprocessing

As our dataset consist of nominal/categorical values only, we will encode the strings into integers which again should simplify our implementation.

In [1140]:
def encode_labels(df):
    import sklearn.preprocessing
    encoder = {}
    for col in df.columns:
        le = sklearn.preprocessing.LabelEncoder()
        le.fit(df[col])
        df[col] = le.transform(df[col])
        encoder[col] = le
    return df, encoder    

mushrooms_encoded_df, encoder = encode_labels(mushrooms_df)

In [1141]:
mushrooms_encoded_df

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,0,3,2,4,0,5,0,0,0,11,...,2,5,5,0,1,1,4,0,1,2
8120,0,5,2,4,0,5,0,0,0,11,...,2,5,5,0,0,1,4,0,4,2
8121,0,2,2,4,0,5,0,0,0,5,...,2,5,5,0,1,1,4,0,1,2
8122,1,3,3,4,0,8,1,0,1,0,...,1,7,7,0,2,1,0,7,4,2


## Dataset Splitting

Before we start with the implementation of our decision tree algorithm we need to prepare our dataset for the training and testing.

First, we divide the dataset into attributes (often called features) and classes (often called targets). Keeping attributes and classes separately is a common practice in many implementations. This should simplify the implementation and make the code understandable.

In [1142]:
X_df = mushrooms_encoded_df.drop('class', axis=1)  # attributes
y_df = mushrooms_encoded_df['class']  # classes
X_array = X_df.to_numpy()
y_array = y_df.to_numpy()

And this is how it looks like.

In [1143]:
print('X =', X_array)
print('y =', y_array)

X = [[5 2 4 ... 2 3 5]
 [5 2 9 ... 3 2 1]
 [0 2 8 ... 3 2 3]
 ...
 [2 2 4 ... 0 1 2]
 [3 3 4 ... 7 4 2]
 [5 2 4 ... 4 1 2]]
y = [1 0 0 ... 0 1 0]


Next, we need to split the attributes and classes into training sets and test sets.

**Exercise:**

Implement the holdout splitting method with shuffling.

In [1144]:
def train_test_split(X, y, test_size=0.2):
    """
    Shuffles the dataset and splits it into training and test sets.
    
    :param X
        attributes
    :param y
        classes
    :param test_size
        float between 0.0 and 1.0 representing the proportion of the dataset to include in the test split
    :return
        train-test splits (X-train, X-test, y-train, y-test)
    """
    ### START CODE HERE ###

    # shuffle X and y, in same order
    permutation = np.random.permutation(X.shape[0])
    X = X[permutation]
    y = y[permutation]

    # split X and y into train and test sets
    split_index = int(X.shape[0] * (1 - test_size))
    X_train = X[:split_index]
    X_test = X[split_index:]
    y_train = y[:split_index]
    y_test = y[split_index:]
    
    ### END CODE HERE ###
    return X_train, X_test, y_train, y_test

Let's split the dataset into training and validation/test set with 67:33 split.

In [1145]:
X_train, X_test, y_train, y_test = train_test_split(X_array, y_array, 0.33)

In [1146]:
print('X_train =', X_train)
print('y_train =', y_train)
print('X_test =', X_test)
print('y_test =', y_test)

X_train = [[5 2 8 ... 7 3 1]
 [5 0 3 ... 3 0 1]
 [2 0 4 ... 2 0 1]
 ...
 [5 0 3 ... 2 0 1]
 [5 0 3 ... 2 0 1]
 [0 2 8 ... 2 2 1]]
y_train = [0 0 0 ... 0 0 0]
X_test = [[2 2 4 ... 3 1 2]
 [2 0 2 ... 2 4 0]
 [5 3 3 ... 2 4 0]
 ...
 [5 3 9 ... 3 3 1]
 [5 3 3 ... 3 4 0]
 [2 2 4 ... 8 1 2]]
y_test = [0 0 0 ... 0 0 0]


## Training

**Exercise:**

Implement an algorithm for fitting (also called training or inducing) a decision tree.

* You have a free hand regarding the generation of candidate splits (also called attribute test conditions).
* Measure the degree of impurity (Gini) to select the best split.

A quick sanity check...

In [1147]:
assert len(X_train) == len(y_train)
assert len(y_train) == 5443
assert len(X_test) == len(y_test)

assert len(y_test) == 2681

In [1148]:
import time
# Use this section to place any "helper" code for the `fit()` function.

### START CODE HERE ###


def calc_gini(classes):
    """
    Calculates the gini impurity of a list of classes.
    
    :param node
        list of classes
    :return
        gini impurity
    """
    gini = 1
    for i in np.unique(classes):
        gini -= (np.sum(classes == i) / classes.size) ** 2
    return gini


# modified function fetched from https://stackoverflow.com/questions/58908934/find-all-combinations-to-split-a-single-list-into-two-lists
def two_partitions(S):
    """
    Finds all possible partitions of a set.
    :param S
        set to partition
    :return
        list of all possible partitions
    """
    res_list = []
    factors = []
    for l in range(0,int(len(S)/2)+1):
        combis = set(itertools.combinations(S,l))
        for c in combis:
            a = (sorted(list(c)))
            b = (sorted(list(set(S)-set(c))))
            if a != [] and b not in factors:
                factors.append(a)
                res_list.append([a, b])
    return res_list


def best_attribute_split(data, classes, attribute):
    """
    Finds the best split for a given attribute.
    
    :param data
        attributes
    :param classes
        classes
    :param attribute
        attribute to split on
    :return
        gini index, split 
    """
    assert attribute < data[0].size # check that attribute is valid
    a = data[:, attribute]
    n = np.unique(data[:, attribute])

    candidate_splits = two_partitions(np.unique(data[:, attribute]))
    min_gini = 1
    best_split = None
    for split in candidate_splits:
        left = np.isin(data[:, attribute], split[0])
        right = np.isin(data[:, attribute], split[1])
        current_gini = (np.sum(left) / left.size) * calc_gini(classes[left]) + (np.sum(right) / right.size) * calc_gini(classes[right])
        if current_gini < min_gini:
            min_gini = current_gini
            best_split = split
    return best_split






        

def info_gain(left, right, p_value):
    """
    Calculates the information gain.
    :param left
        left node
    :param right
        right node
    :param p_value
        P value
    :return
        information gain
    """
    p = left.size/(left.size + right.size)
    return p_value - p * calc_gini(left) - (1 - p) * calc_gini(right)


def find_best_split(data, classes):
    """
    Finds the best question to ask by iterating over every feature / value and calculating the information gain.
    :param data
        attributes
    :param classes
        classes
    :return
        best gain, best question
    """
    t = time.time()
    print('Finding best split...')
    p = calc_gini(classes)
    best_question = None
    best_gain = 0 
    

    for i in range(data[0].size):
        split = best_attribute_split(data, classes, i)

        if split is None:
            continue
        
        question = Question(i, split)

        if question.should_stop(data, classes):
            continue
        
        left_data, right_data, left_classes, right_classes = question.partition(data, classes)
        gain = info_gain(left_classes, right_classes, p)

        if best_gain < gain:
            best_gain = gain
            best_question = question
    print('Best split found in', time.time() - t, 'seconds.')
    return best_gain, best_question


class Question():
    """
    A Question is used to partition a dataset.
    This class records an attribute (e.g., 0 for cap-shape) and a
    split of the possible values for the goiven attribute (e.g. {[0, 1], [2, 3]} for cap-surface).
    """
    def __init__(self, attribute, split):
        self.attribute = attribute
        self.split = split

    def partition_data(self, data):
        """
        Partitions the data based on the question.
        
        :param data
            attributes
        :return
            left data, right data
        """
        left = []
        right = []
        in_left = np.isin(data[:, self.attribute], self.split[0])
        for index, boolean in enumerate(in_left):
            if boolean:
                left.append(data[index])
            else:
                right.append(data[index])
        return np.array(left, dtype=np.int64), np.array(right, dtype=np.int64)
    
    
    def partition(self, data, classes):
        """
        Partitions the data and classes based on the question.

        :param data
            attributes
        :param classes
            classes
        :return
            left data, right data, left classes, right classes
        """
        in_left = np.isin(data[:, self.attribute], self.split[0])
        left_data = []
        right_data = []
        left_classes = []
        right_classes = []

        for index, boolean in enumerate(in_left):
            if boolean:
                left_data.append(data[index])
                left_classes.append(classes[index])
            else:
                right_data.append(data[index])
                right_classes.append(classes[index])
        return np.array(left_data, dtype=np.int64), np.array(right_data, dtype=np.int64), np.array(left_classes, dtype=np.int64), np.array(right_classes, dtype=np.int64)
    

    def should_stop(self, data, classes):
        p = self.partition_data(data)
        return p[0].size == 0 or p[1].size == 0 or calc_gini(classes) == 0


class Node:
    """
    A Node in the decision tree.
    """
    def __init__(self, question, left, right):
        self.question = question
        self.left = left
        self.right = right

    def get_branch(self, row):
        attribute_index = self.question.attribute
        should_go_left = row[attribute_index] in self.question.split[0]
        if should_go_left:
            return self.left
        else:
            return self.right


class Leaf:
    """
    A Leaf node to classify data.
    """
    def __init__(self, classes):
        self.predictions = self._counts(classes)

    def _counts(self, classes):
        counts = {}
        for c in classes:
            if c not in counts:
                counts[c] = 1
            else:
                counts[c] += 1
        return counts


def build_model(data, classes, max_depth=None, depth=0):
    '''
    Builds a decision tree model.
    
    :param data
        attributes
    :param classes
        classes
    :param max_depth
        maximum depth of the decision tree
    :param depth
        current depth of the decision tree
    :return
        decision tree model
    '''
    t = time.time()
    
    gain, question = find_best_split(data, classes)

    if gain == 0 or (max_depth is not None and depth >= max_depth):
        return Leaf(classes)
    
    left_data, right_data, left_classes, right_classes = question.partition(data, classes)

    left_branch = build_model(left_data, left_classes, max_depth, depth+1)
    print("one iteration took " + str(time.time() - t) + " seconds")
    right_branch = build_model(right_data, right_classes, max_depth, depth+1)
    
    return Node(question, left_branch, right_branch)


### END CODE HERE ###

In [1149]:
def fit(X, y):
    """
    Function implementing decision tree induction.
    
    :param X
        attributes
    :param y
        classes
    :return
        trained decision tree (model)
    """
    ### START CODE HERE ### 

    # find the best attribute to split on
    classifier = build_model(X, y, 5)
    
    ### END CODE HERE ### 
    return classifier


In [1150]:
model = fit(X_train, y_train)

Finding best split...
Best split found in 0.7768499851226807 seconds.
Finding best split...
Best split found in 0.2553577423095703 seconds.
Finding best split...
Best split found in 0.0005078315734863281 seconds.
one iteration took 0.257051944732666 seconds
Finding best split...
Best split found in 0.1790757179260254 seconds.
Finding best split...
Best split found in 0.0009031295776367188 seconds.
one iteration took 0.18096303939819336 seconds
Finding best split...
Best split found in 0.1666862964630127 seconds.
Finding best split...
Best split found in 0.0002491474151611328 seconds.
one iteration took 0.1681079864501953 seconds
Finding best split...
Best split found in 0.1641080379486084 seconds.
Finding best split...
Best split found in 0.0018458366394042969 seconds.
one iteration took 0.1669628620147705 seconds
Finding best split...
Best split found in 0.1584770679473877 seconds.
one iteration took 1.7110719680786133 seconds
Finding best split...
Best split found in 0.05337715148925

## Prediction/Deduction

At this moment we should have trained a decision tree (our model). Now we need an algorithm for assigning a class given the attributes and our model.

**Exercise:**

Implement an algorithm deducing class given the attributes and the model.

* `X` is a matrix of attributes of one or more instances for classification.

In [1151]:
# Use this section to place any "helper" code for the `predict()` function.

### START CODE HERE ###

def traverse(node, data):
    """
    Traverses the tree to find the most likely prediction.
    :param node
        current node
    :param data
        data to predict
    :return
        most likely prediction
    """
    if isinstance(node, Leaf):
        # return most likely prediction
        return max(node.predictions, key=node.predictions.get)
    
    node = node.get_branch(data)
    return traverse(node, data)

### END CODE HERE ###

In [1152]:
def predict(X, model):
    """
    Function for generating predictions (classifying) given attributes and model.
    
    :param X
        attributes
    :param model
        model
    :return
        predicted classes (y_hat)
    """
    ### START CODE HERE ###
    y_hat = np.zeros(X.shape[0])
    for i in range(y_hat.size):
        y_hat[i] = traverse(model, X[i])
    ### END CODE HERE ###
    return y_hat

Let's classify the instances of our test set.

In [1153]:
y_hat = predict(X_test, model)

First ten predictions of the test set.

In [1154]:
y_hat[:10]

array([0., 0., 0., 0., 1., 1., 1., 1., 1., 1.])

## Evaluation

Now we would like to assess how well our decision tree classifier performs.

**Exercise:**

Implement a function for calculating the accuracy of your predictions given the ground truth and predictions.

In [1155]:
def evaluate(y_true, y_pred):
    """
    Function calculating the accuracy of the model given the ground truth and predictions.
    
    :param y_true
        true classes
    :param y_pred
        predicted classes
    :return
        accuracy
    """
    ### START CODE HERE ###
    correct = 0
    size = y_pred.size
    for i in range(size):
        if y_pred[i] == y_true[i]:
            correct += 1
    accuracy = correct/size


    ### END CODE HERE ### 
    return accuracy

In [1156]:
accuracy = evaluate(y_test, y_hat)
print('accuracy =', accuracy)

accuracy = 0.9962700484893696


How many items where misclassified?

In [1157]:
print('misclassified =', sum(abs(y_hat - y_test)))

misclassified = 10.0


How balanced is our test set?

In [1158]:
np.bincount(y_test)

array([1364, 1317])

If it's balanced, we don't have to be worried about objectivity of the accuracy metric.

---

Congratulations! At this point, hopefully, you have successufuly implemented a decision tree algorithm able to classify unseen samples with high accuracy.

✌️